In [4]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
import datetime as dt
#Kết nối SV chính (SV 65):
connection_uri = (
    "mssql+pyodbc://PHLHCMHOPPBI02\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)
#Kết nối SV_UAT (SV 64):
connection_uri_UAT = (
    "mssql+pyodbc://PHLHCMHOUPBI01\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn_UAT = sa.create_engine(connection_uri_UAT)
def whole_team_kpi(cutoff):
    #cutoff = input('Nhập kỳ CUTOFF YYYMMDD: ')
    period = cutoff[:6]
    #Lấy bảng:
    py2_query = f"SELECT * FROM [DPO].[DP_K2_CUTOFF] WHERE CUTOFF = {cutoff} " 
    py2 = pd.read_sql(py2_query, conn)
    structure_query = f"SELECT * FROM [dbo].[BSP_TAN_AGENT_STRUCTURE] WHERE CUTOFF = {cutoff}"
    structure = pd.read_sql(structure_query ,conn_UAT)
    individual_kpi_query = f"SELECT * FROM [DPO].[BSP_TAN_INDIVIDUAL_KPI] WHERE Period = {period}"
    individual_kpi = pd.read_sql(individual_kpi_query ,conn_UAT)
    #Tính PY2:
    py2_t = py2
    py2_t = py2_t.groupby(['AGENT NO'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
    kpi_t = individual_kpi
    kpi_t = pd.merge(kpi_t, py2_t, left_on ='Agent_Number', right_on = 'AGENT NO', how = 'left')
    kpi_t = kpi_t.loc[:, kpi_t.columns != 'AGENT NO']
    FM_team = structure
    FM_team = FM_team[FM_team['FM_Code'] != '0']
    FM_team = FM_team[['FM_Code', 'FM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()
    #Tính PY2 từ đây
    FM_team_py2 = pd.merge(FM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
    FM_team_py2 = FM_team_py2.groupby(['FM_Code', 'FM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
    FM_team_py2['PY2'] = np.where(FM_team_py2['Y2 EXPECTED PREM'] == 0, 0, FM_team_py2['Y2 ACTUAL PREM']/FM_team_py2['Y2 EXPECTED PREM'])
    FM_team_py2['PY2'] = np.where(FM_team_py2['PY2'] > 1, 1, FM_team_py2['PY2'])
    FM_team_py2 = FM_team_py2.reset_index()
    FM_team_py2 = FM_team_py2[['FM_Code', 'FM_Name', 'PY2']]
    FM_team_py2['Grade'] = "FM"
    FM_team_py2.rename(columns={'FM_Code':'AL_Code', 'FM_Name':'AL_Name'}, inplace = True)
    #DM Team:
    DM_team = structure
    DM_team = DM_team[DM_team['DM_Code'] != '0']
    DM_team = DM_team[['DM_Code', 'DM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()
    #Tính PY2 từ đây
    DM_team_py2 = pd.merge(DM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
    DM_team_py2 = DM_team_py2.groupby(['DM_Code', 'DM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
    DM_team_py2['PY2'] = np.where(DM_team_py2['Y2 EXPECTED PREM'] == 0, 0, DM_team_py2['Y2 ACTUAL PREM']/DM_team_py2['Y2 EXPECTED PREM'])
    DM_team_py2['PY2'] = np.where(DM_team_py2['PY2'] > 1, 1, DM_team_py2['PY2'])
    DM_team_py2 = DM_team_py2.reset_index()
    DM_team_py2 = DM_team_py2[['DM_Code', 'DM_Name', 'PY2']]
    DM_team_py2['Grade'] = "DM"
    DM_team_py2.rename(columns={'DM_Code':'AL_Code', 'DM_Name':'AL_Name'}, inplace = True)
    #RM Team:
    RM_team = structure
    RM_team = RM_team[RM_team['RM_Code'] != '0']
    RM_team = RM_team[['RM_Code', 'RM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()
    #Tính PY2 từ đây
    RM_team_py2 = pd.merge(RM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
    RM_team_py2 = RM_team_py2.groupby(['RM_Code', 'RM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
    RM_team_py2['PY2'] = np.where(RM_team_py2['Y2 EXPECTED PREM'] == 0, 0, RM_team_py2['Y2 ACTUAL PREM']/RM_team_py2['Y2 EXPECTED PREM'])
    RM_team_py2['PY2'] = np.where(RM_team_py2['PY2'] > 1, 1, RM_team_py2['PY2'])
    RM_team_py2 = RM_team_py2.reset_index()
    RM_team_py2 = RM_team_py2[['RM_Code', 'RM_Name', 'PY2']]
    RM_team_py2['Grade'] = "RM"
    RM_team_py2.rename(columns={'RM_Code':'AL_Code', 'RM_Name':'AL_Name'}, inplace = True)
    #GM Team:
    GM_team = structure
    GM_team = GM_team[GM_team['GM_Code'] != '0']
    GM_team = GM_team[['GM_Code', 'GM_Name', 'AgentNumber', 'AgentName', 'AgentGrade']].drop_duplicates()
    #Tính PY2 từ đây
    GM_team_py2 = pd.merge(GM_team, kpi_t[['Agent_Number', 'Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']], left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left' )
    GM_team_py2 = GM_team_py2.groupby(['GM_Code', 'GM_Name'])[['Y2 ACTUAL PREM', 'Y2 EXPECTED PREM']].sum()
    GM_team_py2['PY2'] = np.where(GM_team_py2['Y2 EXPECTED PREM'] == 0, 0, GM_team_py2['Y2 ACTUAL PREM']/GM_team_py2['Y2 EXPECTED PREM'])
    GM_team_py2['PY2'] = np.where(GM_team_py2['PY2'] > 1, 1, GM_team_py2['PY2'])
    GM_team_py2 = GM_team_py2.reset_index()
    GM_team_py2 = GM_team_py2[['GM_Code', 'GM_Name', 'PY2']]
    GM_team_py2['Grade'] = "GM"
    GM_team_py2.rename(columns={'GM_Code':'AL_Code', 'GM_Name':'AL_Name'}, inplace = True)
    PY2_list = pd.concat([GM_team_py2, RM_team_py2, DM_team_py2, FM_team_py2])
    PY2_list['PY2'] = round(PY2_list['PY2'], 4)
    PY2_list = PY2_list[['AL_Code', 'AL_Name' ,'PY2']]
    manpower = structure
    manpower['Date_On_Set'] = "" #Tạo ra cột trống
    manpower['Date_On_Set']= manpower['Date_On_Set'].str.cat(manpower['CUTOFF'].str[:4] ,sep ="")
    manpower['Date_On_Set']= manpower['Date_On_Set'].str.cat(manpower['CUTOFF'].str[4:6] ,sep ="-")
    manpower['Date_On_Set']= manpower['Date_On_Set'].str.cat(manpower['CUTOFF'].str[6:8] ,sep ="-")
    manpower['Date_On_Set'] = pd.to_datetime(manpower['Date_On_Set']) #nhớ chuyển sang kiểu ngày tháng
    if ('20230531' in manpower['CUTOFF'].values):
        manpower.loc[:,'Start_Month'] = '2023-04-30'
    elif ('20220228' in manpower['CUTOFF'].values):
        manpower.loc[:,'Start_Month'] = '2022-01-28'
    elif ('20220429' in manpower['CUTOFF'].values):
        manpower.loc[:,'Start_Month'] = '2022-03-31'
    elif ('20220531' in manpower['CUTOFF'].values):
        manpower.loc[:,'Start_Month'] = '2022-04-30' 
    elif ('20240229' in manpower['CUTOFF'].values):
        manpower.loc[:,'Start_Month'] = '2024-01-27'
    else: 
        manpower['Start_Month'] = (manpower['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
    inforce = manpower
    #FM Inforce
    fm_inforce = inforce
    fm_inforce = fm_inforce[['FM_Code', 'FM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
    fm_inforce = fm_inforce[(fm_inforce['Agent_Status'] !='Terminated') & (fm_inforce['FM_Code'] != '0')]
    fm_inforce = pd.merge(fm_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
    fm_inforce = fm_inforce.loc[:,fm_inforce.columns != 'Agent_Number']
    segment = fm_inforce['Segmentation'].drop_duplicates().to_list()
    fm_inforce_sum = fm_inforce
    fm_inforce_sum = fm_inforce_sum.groupby(['FM_Code','FM_Name','Segmentation']).size().unstack(fill_value=0)
    fm_inforce_sum = fm_inforce_sum.reset_index()
    fm_inforce_sum['Grade'] = "FM"
    fm_inforce_sum = fm_inforce_sum.rename(columns={'FM_Code': 'AL_Code', 'FM_Name': 'AL_Name'})
    #DM Inforce:
    DM_inforce = inforce
    DM_inforce = DM_inforce[['DM_Code', 'DM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
    DM_inforce = DM_inforce[(DM_inforce['Agent_Status'] !='Terminated') & (DM_inforce['DM_Code'] != '0')]
    DM_inforce = pd.merge(DM_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
    DM_inforce = DM_inforce.loc[:,DM_inforce.columns != 'Agent_Number']
    #
    DM_inforce_sum = DM_inforce
    DM_inforce_sum = DM_inforce_sum.groupby(['DM_Code','DM_Name', 'Segmentation']).size().unstack(fill_value=0)
    DM_inforce_sum = DM_inforce_sum.reset_index()
    DM_inforce_sum['Grade'] = "DM"
    DM_inforce_sum = DM_inforce_sum.rename(columns={'DM_Code': 'AL_Code', 'DM_Name': 'AL_Name'})
    #RM Inforce:
    RM_inforce = inforce
    RM_inforce = RM_inforce[['RM_Code', 'RM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
    RM_inforce = RM_inforce[(RM_inforce['Agent_Status'] !='Terminated') & (RM_inforce['RM_Code'] != '0')]
    RM_inforce = pd.merge(RM_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
    RM_inforce = RM_inforce.loc[:,RM_inforce.columns != 'Agent_Number']
    #
    RM_inforce_sum = RM_inforce
    RM_inforce_sum = RM_inforce_sum.groupby(['RM_Code','RM_Name', 'Segmentation']).size().unstack(fill_value=0)
    RM_inforce_sum = RM_inforce_sum.reset_index()
    RM_inforce_sum['Grade'] = "RM"
    RM_inforce_sum = RM_inforce_sum.rename(columns={'RM_Code': 'AL_Code', 'RM_Name': 'AL_Name'})
    #GM Inforce:
    GM_inforce = inforce
    GM_inforce = GM_inforce[['GM_Code', 'GM_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status']]
    GM_inforce = GM_inforce[(GM_inforce['Agent_Status'] !='Terminated') & (GM_inforce['GM_Code'] != '0')]
    GM_inforce = pd.merge(GM_inforce, kpi_t[['Agent_Number', 'Segmentation']], left_on = 'AgentNumber', right_on='Agent_Number', how ='left')
    GM_inforce = GM_inforce.loc[:,GM_inforce.columns != 'Agent_Number']
    #
    GM_inforce_sum = GM_inforce
    GM_inforce_sum = GM_inforce_sum.groupby(['GM_Code','GM_Name', 'Segmentation']).size().unstack(fill_value=0)
    GM_inforce_sum = GM_inforce_sum.reset_index()
    GM_inforce_sum['Grade'] = "GM"
    GM_inforce_sum = GM_inforce_sum.rename(columns={'GM_Code': 'AL_Code', 'GM_Name': 'AL_Name'})
    Manpower_Inforce = pd.concat([fm_inforce_sum, DM_inforce_sum, RM_inforce_sum, GM_inforce_sum ])
    AL_inforce = inforce
    AL_lst = ['FM', 'DM', 'RM', 'GM']
    #
    FM_count = AL_inforce[['FM_Code', 'FM_Name', 'AgentGrade']]
    FM_count = FM_count[FM_count['FM_Code'] != '0']
    FM_count = FM_count.groupby(['FM_Code', 'FM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    FM_count = FM_count.reset_index()
    for col in AL_lst:
        if col not in FM_count.columns:
            FM_count[col] = 0
    FM_count = FM_count.rename(columns={'FM_Code':'AL_Code', 'FM_Name': 'AL_Name'})
    FM_count['Grade'] = 'FM'
    #
    DM_count = AL_inforce[['DM_Code', 'DM_Name', 'AgentGrade']]
    DM_count = DM_count[DM_count['DM_Code'] != '0']
    DM_count = DM_count.groupby(['DM_Code', 'DM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    DM_count = DM_count.reset_index()
    for col in AL_lst:
        if col not in DM_count.columns:
            DM_count[col] = 0
    DM_count = DM_count.rename(columns={'DM_Code':'AL_Code', 'DM_Name': 'AL_Name'})
    DM_count['Grade'] = 'DM'
    #
    RM_count = AL_inforce[['RM_Code', 'RM_Name', 'AgentGrade']]
    RM_count = RM_count[RM_count['RM_Code'] != '0']
    RM_count = RM_count.groupby(['RM_Code', 'RM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    RM_count = RM_count.reset_index()
    for col in AL_lst:
        if col not in RM_count.columns:
            RM_count[col] = 0
    RM_count = RM_count.rename(columns={'RM_Code':'AL_Code', 'RM_Name': 'AL_Name'})
    RM_count['Grade'] = 'RM'
    #
    GM_count = AL_inforce[['GM_Code', 'GM_Name', 'AgentGrade']]
    GM_count = GM_count[GM_count['GM_Code'] != '0']
    GM_count = GM_count.groupby(['GM_Code', 'GM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    GM_count = GM_count.reset_index()
    for col in AL_lst:
        if col not in GM_count.columns:
            GM_count[col] = 0
    GM_count = GM_count.rename(columns={'GM_Code':'AL_Code', 'GM_Name': 'AL_Name'})
    GM_count['Grade'] = 'GM'
    GM_FM = pd.concat([FM_count, DM_count, RM_count, GM_count])
    Inforce_MP = pd.merge(Manpower_Inforce, GM_FM, on = ['AL_Code', 'AL_Name'], how = 'left')
    Inforce_MP = Inforce_MP[['AL_Code', 'AL_Name', 'AL', 'AL Trainee', 'Agent Trainee', 'FC_x', 'SFC', 'Grade_x', 'PHL', 'FM', 'DM', 'RM', 'GM']].\
    rename(columns={'Grade_x':'Grade', 'FC_x':'FC'})
    recruitment = manpower
    recruitment = recruitment[(recruitment['Date_Appointed'] >= recruitment['Start_Month']) & 
                         (recruitment['Date_Appointed'] <= recruitment['Date_On_Set'])]
    #Recruitment:
    FM_recruit = recruitment
    FM_recruit = FM_recruit[['FM_Code', 'FM_Name', 'AgentGrade']]
    FM_recruit = FM_recruit[FM_recruit['FM_Code'] != '0']
    FM_recruit = FM_recruit.groupby(['FM_Code', 'FM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    for col in AL_lst:
        if col not in FM_recruit.columns:
            FM_recruit[col] = 0
    FM_recruit = FM_recruit.reset_index()
    FM_recruit['Grade'] = "FM"
    FM_recruit = FM_recruit.rename(columns={'FM_Code': 'AL_Code', 'FM_Name':'AL_Name'})
    DM_recruit = recruitment
    DM_recruit = DM_recruit[['DM_Code', 'DM_Name', 'AgentGrade']]
    DM_recruit = DM_recruit[DM_recruit['DM_Code'] != '0']
    DM_recruit = DM_recruit.groupby(['DM_Code', 'DM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    for col in AL_lst:
        if col not in DM_recruit.columns:
            DM_recruit[col] = 0
    DM_recruit = DM_recruit.reset_index()
    DM_recruit['Grade'] = "DM"
    DM_recruit = DM_recruit.rename(columns={'DM_Code': 'AL_Code', 'DM_Name':'AL_Name'})
    RM_recruit = recruitment
    RM_recruit = RM_recruit[['RM_Code', 'RM_Name', 'AgentGrade']]
    RM_recruit = RM_recruit[RM_recruit['RM_Code'] != '0']
    RM_recruit = RM_recruit.groupby(['RM_Code', 'RM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    for col in AL_lst:
        if col not in RM_recruit.columns:
            RM_recruit[col] = 0
    RM_recruit = RM_recruit.reset_index()
    RM_recruit['Grade'] = "RM"
    RM_recruit = RM_recruit.rename(columns={'RM_Code': 'AL_Code', 'RM_Name':'AL_Name'})
    GM_recruit = recruitment
    GM_recruit = GM_recruit[['GM_Code', 'GM_Name', 'AgentGrade']]
    GM_recruit = GM_recruit[GM_recruit['GM_Code'] != '0']
    GM_recruit = GM_recruit.groupby(['GM_Code', 'GM_Name', 'AgentGrade']).size().unstack(fill_value=0)
    for col in AL_lst:
        if col not in GM_recruit.columns:
            GM_recruit[col] = 0
    GM_recruit = GM_recruit.reset_index()
    GM_recruit['Grade'] = "GM"
    GM_recruit = GM_recruit.rename(columns={'GM_Code': 'AL_Code', 'GM_Name':'AL_Name'})
    monthly_recruit = pd.concat([FM_recruit, DM_recruit, RM_recruit, GM_recruit])
    recruit_active = pd.merge(recruitment, individual_kpi[['Agent_Number', 'Active Agent']], left_on ='AgentNumber', right_on = 'Agent_Number', how ='left')
    recruit_active = recruit_active[recruit_active['Active Agent'] == '1']
    recruit_active = recruit_active.loc[:, recruit_active.columns != 'Agent_Number']
    #Lấy ra FM:
    FM_active = recruit_active
    FM_active = FM_active[['FM_Code', 'AgentNumber', 'Active Agent']]
    FM_active = FM_active[FM_active['FM_Code'] != '0']
    FM_active = FM_active.groupby(['FM_Code'])['Active Agent'].count()
    FM_active = FM_active.reset_index()
    FM_active.rename(columns={'FM_Code':'AL_Code'}, inplace = True)
    #Lấy ra DM:
    DM_active = recruit_active
    DM_active = DM_active[['DM_Code', 'AgentNumber', 'Active Agent']]
    DM_active = DM_active[DM_active['DM_Code'] != '0']
    DM_active = DM_active.groupby(['DM_Code'])['Active Agent'].count()
    DM_active = DM_active.reset_index()
    DM_active.rename(columns={'DM_Code':'AL_Code'}, inplace = True)
    #Lấy ra RM:
    RM_active = recruit_active
    RM_active = RM_active[['RM_Code', 'AgentNumber', 'Active Agent']]
    RM_active = RM_active[RM_active['RM_Code'] != '0']
    RM_active = RM_active.groupby(['RM_Code'])['Active Agent'].count()
    RM_active = RM_active.reset_index()
    RM_active.rename(columns={'RM_Code':'AL_Code'}, inplace = True)
    #Lấy ra GM:
    GM_active = recruit_active
    GM_active = GM_active[['GM_Code', 'AgentNumber', 'Active Agent']]
    GM_active = GM_active[GM_active['GM_Code'] != '0']
    GM_active = GM_active.groupby(['GM_Code'])['Active Agent'].count()
    GM_active = GM_active.reset_index()
    GM_active.rename(columns={'GM_Code':'AL_Code'}, inplace = True)
    active_list = pd.concat([FM_active, DM_active, RM_active, GM_active])
    active_list.rename(columns={'Active Agent': 'Recruit Active'}, inplace = True)
    recruitment_full = pd.merge(monthly_recruit, active_list, on='AL_Code', how = 'left')
    recruitment_full.fillna(0, inplace = True)
    for col in ['FC', 'FM','DM','RM','GM','Recruit Active']:
        recruitment_full[col] = recruitment_full[col].astype('int')
    recruitment_full['Recruit AL'] =0 
    for col in ['FC', 'FM','DM','RM','GM']:
        recruitment_full['Recruit AL'] += recruitment_full[col]
    recruitment_full.rename(columns={'FC': 'Recruit FC'}, inplace =True)
    recruitment_full['Recruit Total'] = recruitment_full['Recruit FC'] + recruitment_full['Recruit AL']
    recruitment_full['ActiveRatioNewRecruit'] = np.where(recruitment_full['Recruit Total'] == 0,0,
                                    round(recruitment_full['Recruit Active'] / recruitment_full['Recruit Total'], 4))
    for i in ['FM', 'DM', 'RM', 'GM']:
        recruitment_full.rename(columns={i:'Recruit_'+i}, inplace=True)
    terminate_active = manpower
    terminate_active = terminate_active[terminate_active['Terminated_date'] != '0']
    terminate_active['Terminated_date'] = pd.to_datetime(terminate_active['Terminated_date'])
    terminate_active = terminate_active[(terminate_active['Terminated_date'] >= terminate_active['Start_Month']) &
                   (terminate_active['Terminated_date'] <= terminate_active['Date_On_Set']) ]
    terminate_agent = terminate_active
    terminate_agent['AgentGrade'] = np.where(terminate_agent['AgentGrade'].isin(['FC', 'FM', 'DM', 'RM', 'GM']), terminate_agent['AgentGrade'],
                                        'FC')
    #Tính FM:
    FM_ter = terminate_agent
    FM_ter = FM_ter[['FM_Code', 'AgentNumber', 'AgentGrade']]
    FM_ter= FM_ter[FM_ter['FM_Code'] != '0']
    FM_ter['AL Ter'] = np.where(FM_ter['AgentGrade'] == 'FC', 0, 1)
    FM_ter['FC Ter'] = np.where(FM_ter['AgentGrade'] == 'FC', 1, 0)
    FM_ter = FM_ter.groupby(['FM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'FM_Code': 'AL_Code'})
    #Tính DM:
    DM_ter = terminate_agent
    DM_ter = DM_ter[['DM_Code', 'AgentNumber', 'AgentGrade']]
    DM_ter= DM_ter[DM_ter['DM_Code'] != '0']
    DM_ter['AL Ter'] = np.where(DM_ter['AgentGrade'] == 'FC', 0, 1)
    DM_ter['FC Ter'] = np.where(DM_ter['AgentGrade'] == 'FC', 1, 0)
    DM_ter = DM_ter.groupby(['DM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'DM_Code': 'AL_Code'})
    #Tính RM:
    RM_ter = terminate_agent
    RM_ter = RM_ter[['RM_Code', 'AgentNumber', 'AgentGrade']]
    RM_ter= RM_ter[RM_ter['RM_Code'] != '0']
    RM_ter['AL Ter'] = np.where(RM_ter['AgentGrade'] == 'FC', 0, 1)
    RM_ter['FC Ter'] = np.where(RM_ter['AgentGrade'] == 'FC', 1, 0)
    RM_ter = RM_ter.groupby(['RM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'RM_Code': 'AL_Code'})
    #Tính GM:
    GM_ter = terminate_agent
    GM_ter = GM_ter[['GM_Code', 'AgentNumber', 'AgentGrade']]
    GM_ter= GM_ter[GM_ter['GM_Code'] != '0']
    GM_ter['AL Ter'] = np.where(GM_ter['AgentGrade'] == 'FC', 0, 1)
    GM_ter['FC Ter'] = np.where(GM_ter['AgentGrade'] == 'FC', 1, 0)
    GM_ter = GM_ter.groupby(['GM_Code'])[['AL Ter', 'FC Ter']].sum().reset_index().rename(columns={'GM_Code': 'AL_Code'})
    ter_list = pd.concat([FM_ter, DM_ter, RM_ter, GM_ter])
    #tổng hợp giai đoạn 1:
    agent_info = structure
    agent_info = agent_info[agent_info['AgentGrade'].isin(['FM', 'DM', 'RM', 'GM'])]
    agent_info['Period'] = agent_info['CUTOFF'].str[:6]
    agent_info = agent_info[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status', 'Date_Appointed']]
    #PY2:
    al_py2 = pd.merge(agent_info, PY2_list, left_on='AgentNumber', right_on='AL_Code', how= 'left')
    al_py2 = al_py2[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status','Date_Appointed', 'PY2']]
    al_py2
    #MP:
    al_mp = pd.merge(al_py2, Inforce_MP, left_on='AgentNumber', right_on='AL_Code', how= 'left' )
    al_mp = al_mp.loc[:, ~al_mp.columns.isin(['AL_Code','AL_Name'])]
    al_mp['MP_Total'] = al_mp['AL'] + al_mp['FC']
    al_mp = al_mp[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status','Date_Appointed', 'PY2',
        'GM', 'RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC']]
    al_mp
    #Recruit:
    recruit_al = pd.merge(al_mp, recruitment_full, left_on='AgentNumber', right_on='AL_Code', how= 'left')
    recruit_al = recruit_al.loc[:,~recruit_al.columns.isin(['Grade', 'AL_Code', 'AL_Name'])]
    recruit_al
    #ter:
    ter_al = pd.merge(recruit_al, ter_list, left_on='AgentNumber', right_on='AL_Code', how= 'left')
    ter_al = ter_al.loc[:, ~ter_al.columns.isin(['AL_Code'])]
    ter_al['Total Agent Ter'] = ter_al['AL Ter'] + ter_al['FC Ter']
    ter_al['%Ter'] = np.where((ter_al['MP_Total'] + ter_al['Total Agent Ter'] ==0), 0, 
                round(ter_al['Total Agent Ter'] / (ter_al['MP_Total'] + ter_al['Total Agent Ter']), 4))
    ter_al = ter_al[['Period', 'AgentNumber', 'AgentName', 'AgentGrade', 'Agent_Status','Date_Appointed', 'PY2', 
        'GM', 'RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC',
        'Recruit_RM', 'Recruit_DM', 'Recruit_FM', 'Recruit AL', 'Recruit FC', 'Recruit Total', 'Recruit Active', 'ActiveRatioNewRecruit',
        'AL Ter','FC Ter', 'Total Agent Ter', '%Ter']]
    cc_kpi = individual_kpi
    cc_kpi = cc_kpi[['Agent_Number', 'CC(+)', 'CC(-)', 'CC']]
    for col in ['CC(+)', 'CC(-)', 'CC']:
        cc_kpi[col] = cc_kpi[col].astype('int')
    cc_kpi = pd.merge(structure, cc_kpi, left_on = 'AgentNumber', right_on ='Agent_Number', how='left')
    #FM:
    FM_cc = cc_kpi[['FM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
    FM_cc = FM_cc[FM_cc['FM_Code'] != '0']
    FM_cc = FM_cc.groupby(['FM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
    FM_cc = FM_cc.rename(columns={'FM_Code':'AL_Code'})
    #DM:
    DM_cc = cc_kpi[['DM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
    DM_cc = DM_cc[DM_cc['DM_Code'] != '0']
    DM_cc = DM_cc.groupby(['DM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
    DM_cc = DM_cc.rename(columns={'DM_Code':'AL_Code'})
    #RM:
    RM_cc = cc_kpi[['RM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
    RM_cc = RM_cc[RM_cc['RM_Code'] != '0']
    RM_cc = RM_cc.groupby(['RM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
    RM_cc = RM_cc.rename(columns={'RM_Code':'AL_Code'})
    #GM:
    GM_cc = cc_kpi[['GM_Code', 'AgentNumber', 'CC(+)', 'CC(-)', 'CC' ]]
    GM_cc = GM_cc[GM_cc['GM_Code'] != '0']
    GM_cc = GM_cc.groupby(['GM_Code'])[['CC(+)', 'CC(-)', 'CC']].sum().reset_index()
    GM_cc = GM_cc.rename(columns={'GM_Code':'AL_Code'})
    agent_cc = pd.concat([FM_cc, DM_cc, RM_cc, GM_cc])
    al_cc = pd.merge(ter_al, agent_cc, left_on='AgentNumber', right_on ='AL_Code', how = 'left')
    al_cc = al_cc.loc[:, al_cc.columns != 'AL_Code']
    policy_kpi = individual_kpi
    policy_kpi = policy_kpi[['Agent_Number', 'Base', 'Rider']]
    policy_kpi = pd.merge(structure, policy_kpi, left_on = 'AgentNumber', right_on ='Agent_Number', how='left')
    policy_kpi.fillna(value=0, inplace=True)
    for i in ['Base', 'Rider']:
        policy_kpi[i] = policy_kpi[i].astype('int')
    #FM:
    FM_policy = policy_kpi[['FM_Code', 'AgentNumber', 'Base', 'Rider']]
    FM_policy = FM_policy[FM_policy['FM_Code'] != '0']
    FM_policy = FM_policy.groupby(['FM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'FM_Code':'AL_Code'})
    #DM:
    DM_policy = policy_kpi[['DM_Code', 'AgentNumber', 'Base', 'Rider']]
    DM_policy = DM_policy[DM_policy['DM_Code'] != '0']
    DM_policy = DM_policy.groupby(['DM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'DM_Code':'AL_Code'})
    #RM:
    RM_policy = policy_kpi[['RM_Code', 'AgentNumber', 'Base', 'Rider']]
    RM_policy = RM_policy[RM_policy['RM_Code'] != '0']
    RM_policy = RM_policy.groupby(['RM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'RM_Code':'AL_Code'})
    #GM:
    GM_policy = policy_kpi[['GM_Code', 'AgentNumber', 'Base', 'Rider']]
    GM_policy = GM_policy[GM_policy['GM_Code'] != '0']
    GM_policy = GM_policy.groupby(['GM_Code'])[['Base','Rider']].sum().reset_index().rename(columns={'GM_Code':'AL_Code'})
    #Tổng hợp kết quả
    al_policy = pd.concat([FM_policy, DM_policy, RM_policy, GM_policy])
    al_policy = pd.merge(al_cc, al_policy, left_on = 'AgentNumber', right_on ='AL_Code', how='left')
    for col in ['Base', 'Rider']:
        al_policy[col] = al_policy[col].astype('int')
    al_policy['%Rider/Base'] = np.where(al_policy['Rider'] == 0, 0, round(al_policy['Rider']/al_policy['Base'], 4))
    al_policy = al_policy.loc[:, ~al_policy.columns.isin(['AL_Code'])]
    aa_al = cc_kpi[cc_kpi['CC'] > 0]
    aa_al['Active Agent'] = 1
    #FM:
    FM_aa = aa_al[['FM_Code', 'Agent_Number', 'Active Agent']]
    FM_aa = FM_aa[FM_aa['FM_Code'] !='0']
    FM_aa = FM_aa.groupby(['FM_Code'])['Active Agent'].sum().reset_index().rename(columns={'FM_Code':'AL_Code'})
    #DM:
    DM_aa = aa_al[['DM_Code', 'Agent_Number', 'Active Agent']]
    DM_aa = DM_aa[DM_aa['DM_Code'] !='0']
    DM_aa = DM_aa.groupby(['DM_Code'])['Active Agent'].sum().reset_index().rename(columns={'DM_Code':'AL_Code'})
    #RM:
    RM_aa = aa_al[['RM_Code', 'Agent_Number', 'Active Agent']]
    RM_aa = RM_aa[RM_aa['RM_Code'] !='0']
    RM_aa = RM_aa.groupby(['RM_Code'])['Active Agent'].sum().reset_index().rename(columns={'RM_Code':'AL_Code'})
    #GM:
    GM_aa = aa_al[['GM_Code', 'Agent_Number', 'Active Agent']]
    GM_aa = GM_aa[GM_aa['GM_Code'] !='0']
    GM_aa = GM_aa.groupby(['GM_Code'])['Active Agent'].sum().reset_index().rename(columns={'GM_Code':'AL_Code'})
    #Tổng hợp:
    sum_aa = pd.concat([FM_aa, DM_aa, RM_aa, GM_aa])
    al_aa_stage = pd.merge(al_policy, sum_aa, left_on = 'AgentNumber', right_on ='AL_Code', how ='left')
    al_aa_stage = al_aa_stage.loc[:, ~al_aa_stage.columns.isin(['AL_Code'])]
    al_aa_stage['Active Agent'].fillna(value =0, inplace= True)
    al_aa_stage['Activity Ratio'] = np.where(al_aa_stage['MP_Total']  == 0, 0,
    round(al_aa_stage['Active Agent'] / al_aa_stage['MP_Total'], 4))
    afyp_aa = individual_kpi[['Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
    afyp_aa = pd.merge(structure, afyp_aa, left_on = 'AgentNumber', right_on = 'Agent_Number', how = 'left')
    #FM:
    FM_afyp = afyp_aa[['FM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
    FM_afyp = FM_afyp[FM_afyp['FM_Code'] != '0']
    for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
        FM_afyp[col] = FM_afyp[col].astype('float')
    FM_afyp = FM_afyp.groupby(['FM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'FM_Code': 'AL_Code'})
    #DM:
    DM_afyp = afyp_aa[['DM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
    DM_afyp = DM_afyp[DM_afyp['DM_Code'] != '0']
    for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
        DM_afyp[col] = DM_afyp[col].astype('float')
    DM_afyp = DM_afyp.groupby(['DM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'DM_Code': 'AL_Code'})
    #RM:
    RM_afyp = afyp_aa[['RM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
    RM_afyp = RM_afyp[RM_afyp['RM_Code'] != '0']
    for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
        RM_afyp[col] = RM_afyp[col].astype('float')
    RM_afyp = RM_afyp.groupby(['RM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'RM_Code': 'AL_Code'})
    #GM:
    GM_afyp = afyp_aa[['GM_Code', 'Agent_Number', 'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']]
    GM_afyp = GM_afyp[GM_afyp['GM_Code'] != '0']
    for col in ['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']:
        GM_afyp[col] = GM_afyp[col].astype('float')
    GM_afyp = GM_afyp.groupby(['GM_Code'])[['AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)']].sum().reset_index().rename(columns={'GM_Code': 'AL_Code'})
    sum_afyp = pd.concat([FM_afyp, DM_afyp, RM_afyp, GM_afyp])
    afyp_stage = pd.merge(al_aa_stage, sum_afyp, left_on ='AgentNumber', right_on = 'AL_Code', how='left')
    afyp_stage = afyp_stage.loc[:, ~afyp_stage.columns.isin(['AL_Code'])]
    afyp_stage['AFYP'] = afyp_stage['AFYP(+)'] + afyp_stage['AFYP(-)']
    afyp_stage['FYP'] = afyp_stage['FYP(+)'] + afyp_stage['FYP(-)']
    case_size = afyp_stage
    case_size['Case Size'] = np.where(case_size['CC'] == 0, 0, round(case_size['AFYP'] / case_size['CC'], 2))
    case_size['Cases_per_Active'] = np.where(case_size['Active Agent'] ==0, 0, 
                                         round(case_size['CC']/ case_size['Active Agent'], 2))
    for col in ['GM','RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC', 'Recruit_RM', 'Recruit_DM', 'Recruit_FM',
     'Recruit AL', 'Recruit FC', 'Recruit Total', 'Recruit Active', 'ActiveRatioNewRecruit', 'AL Ter', 'FC Ter', 'Total Agent Ter',
     '%Ter', 'CC(+)', 'CC(-)', 'CC', 'Base', 'Rider', '%Rider/Base', 'Active Agent', 'Activity Ratio',
     'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)', 'AFYP', 'FYP', 'Case Size', 'Cases_per_Active']:
        case_size[col] = case_size[col].astype('float')
    case_size.fillna(value=0, inplace= True)
    for col in ['GM','RM', 'DM', 'FM', 'AL', 'FC', 'MP_Total', 'AL Trainee', 'Agent Trainee', 'SFC', 'Recruit_RM', 'Recruit_DM', 'Recruit_FM',
     'Recruit AL', 'Recruit FC', 'Recruit Total', 'Recruit Active', 'ActiveRatioNewRecruit', 'AL Ter', 'FC Ter', 'Total Agent Ter',
     '%Ter', 'CC(+)', 'CC(-)', 'CC', 'Base', 'Rider', '%Rider/Base', 'Active Agent', 'Activity Ratio',
     'AFYP(+)', 'AFYP(-)', 'FYP(+)', 'FYP(-)', 'AFYP', 'FYP', 'Case Size', 'Cases_per_Active']:
        case_size[col] = case_size[col].astype('str')
        case_size[col] = np.where(case_size[col] == '0.0', '0', case_size[col])
        case_size[col] = [i.replace('.0','') for i in case_size[col]]
    final_team_kpi = case_size
    insert_query = """\
    INSERT INTO [DPO].[BSP_TAN_WHOLE_TEAM_KPI] ([Period],[AgentNumber],[AgentName],[AgentGrade],[Agent_Status],[Date_Appointed],
    [PY2],[GM],[RM],[DM],[FM],[AL],[FC],[MP_Total],[AL Trainee],[Agent Trainee],[SFC],[Recruit_RM],[Recruit_DM],[Recruit_FM],[Recruit AL],
    [Recruit FC],[Recruit Total],[Recruit Active],[ActiveRatioNewRecruit],[AL Ter],[FC Ter],[Total Agent Ter],[%Ter],[CC(+)],
    [CC(-)],[CC],[Base],[Rider],[%Rider/Base],[Active Agent],[Activity Ratio],[AFYP(+)],[AFYP(-)],[AFYP],[FYP(+)],[FYP(-)],
    [FYP],[Case Size],[Cases_per_Active]
    )
      VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """
    #3.Tạo Connection với SQL
    conn_toSQL = pyo.connect(driver='{ODBC Driver 17 for SQL Server}', host = 'PHLHCMHOUPBI01\PBIRS', database = 'PowerBI', trusted_connection = 'yes',
                        ansi=True,autocommit=True) #LƯU Ý: ANSI GIỮ FONT TIẾNG VIỆT
    cursor = conn_toSQL.cursor()
    for index, row in final_team_kpi.iterrows():
        cursor.execute(insert_query,
    row['Period'],
    row['AgentNumber'],
    row['AgentName'],
    row['AgentGrade'],
    row['Agent_Status'],
    row['Date_Appointed'],
    row['PY2'],
    row['GM'],
    row['RM'],
    row['DM'],
    row['FM'],
    row['AL'],
    row['FC'],
    row['MP_Total'],
    row['AL Trainee'],
    row['Agent Trainee'],
    row['SFC'],
    row['Recruit_RM'],
    row['Recruit_DM'],
    row['Recruit_FM'],
    row['Recruit AL'],
    row['Recruit FC'],
    row['Recruit Total'],
    row['Recruit Active'],
    row['ActiveRatioNewRecruit'],
    row['AL Ter'],
    row['FC Ter'],
    row['Total Agent Ter'],
    row['%Ter'],
    row['CC(+)'],
    row['CC(-)'],
    row['CC'],
    row['Base'],
    row['Rider'],
    row['%Rider/Base'],
    row['Active Agent'],
    row['Activity Ratio'],
    row['AFYP(+)'],
    row['AFYP(-)'],
    row['FYP(+)'],
    row['FYP(-)'],
    row['AFYP'],
    row['FYP'],
    row['Case Size'],
    row['Cases_per_Active']
    )
    conn_toSQL.commit()
    cursor.close()
    return final_team_kpi.shape[0]

In [3]:
cutoff_list = ['20220127',
'20220228',
'20220329',
'20220429',
'20220531',
'20220630',
'20220731',
'20220831',
'20220930',
'20221031',
'20221130',
'20221231',
'20230131',
'20230228',
'20230331',
'20230429',
'20230531',
'20230630',
'20230731',
'20230831',
'20230930',
'20231031',
'20231130',
'20231231',
'20240126'
]
for cutoff in cutoff_list:
        whole_team_kpi(cutoff)

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_15156\3343152285.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  terminate_active['Terminated_date'] = pd.to_datetime(terminate_active['Terminated_date'])
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_15156\3343152285.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agent_info['Period'] = agent_info['CUTOFF'].str[:6]
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_15156\3343152285.py:364: SettingWithCopyWarning: 
A value is trying to be set on a copy

ValueError: cannot convert float NaN to integer